## Importing libraries

In [1]:
import sys
import json
import nanohubuidl.teleport as t
from nanohubuidl.rappture import RapptureBuilder
from nanohubuidl.material import MaterialBuilder
from nanohubuidl.plotly import PlotlyBuilder
from nanohubuidl.material import MaterialContent
from nanohubuidl.teleport import TeleportElement, TeleportContent
from nanohubuidl.app import AppBuilder

from nanohubuidl.nanohub import Auth
TOOLNAME="kronig_penney"
STATE_LOADER_STATUS = "loader_status"
STATE_LOADER_OPEN = "loader_open"
STATE_ERROR_STATUS = "error_status"
STATE_ERROR_OPEN = "error_open"


## Creating the project and adding GUI variables 

In [2]:
Project = t.TeleportProject("Periodic Potential Lab")
Component = Project.root
Component.addStateVariable("DrawerIsVisible", {"type":"boolean", "defaultValue": True})
Component.addStateVariable("open_results", {"type":"boolean", "defaultValue": False})
Component.addStateVariable("open_params", {"type":"boolean", "defaultValue": True})

## Getting session information

In [3]:
import os
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]

## Creating Error

In [4]:
ErrorMessage = RapptureBuilder.Error( 
    Component, 
    error_status = STATE_ERROR_STATUS,
    error_open = STATE_ERROR_OPEN,
)

## Creating Loader

In [5]:
Loader = RapptureBuilder.Loader( 
    Component, 
    loader_status = STATE_LOADER_STATUS,
    loader_open = STATE_LOADER_OPEN,
)

## Schema

In [6]:
RapptureBuilder.buildSchema(
    Project,
    Component,
    url = "https://nanohub.org/api/tools",
    toolname = TOOLNAME
)
Component.addPropVariable("onLoadSchema", {"type":"func", 'defaultValue' : '(e)=>{e.setState({"'+ STATE_LOADER_OPEN +'":false})}'})    

## Authentication

In [7]:
Login, CLogin = Auth.Session(
    Project,
    Component,
    sessiontoken = auth_data["sessiontoken"],
    sessionnum = auth_data["sessionnum"],
    url = "https://nanohub.org/api/developer/oauth/token",   
)
Login.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN, "newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS, "newState": '$e'},
]
Login.content.events["onAuth"] = [
    { "type": "propCall2", "calls": "buildSchema", "args": ['self'] }            
]


In [8]:
Drawer = MaterialBuilder.Drawer(Project, state="DrawerIsVisible", variant="persistent" )

In [9]:
def StepIcon():
    return "M 0 22 h 5 v -20 h 5 v 20 h 5 v -20 h5 v 20 h 5"

def SinosoidIcon():
    return "M 0 12 q 6 -15 12 0 q 6 15 12 0"

def TriangularIcon():
    return "M0 22 h 7 l 5 -20 l 6 20 h 7"

def ParabolicIcon():
    return "M 0 5 q 12 36 24 0"

def CoulombIcon():
    return "M 0 2 q 2 10 3 20 c 0 -25 16 -25 16 0 q 0 -10 3 -20"

def ExponentialIcon():
    return "M 0 22 q 8 0 11 -20 q 7 20 11 20"

def GaussianIcon():
    return "M 0 5 q 13 -12 22 17"

def PolynomialIcon():
    return "M 0 18 q 8 10 22 -17"
    

In [10]:
SETTINGS = {
    'pot_type': { 
        'type': 'IconList','default_value': 'KP', 'units': None, 'label': 'Type of periodic potential',  
        'description': 'Periodic potential assumed in the infinite lattice.', 
        'options': {
            'KP': { 'name':'Step Well', "icon": StepIcon()},
            'Sine': { 'name':'Sinosoidal Potential', "icon": SinosoidIcon()},
            'Tri': { 'name':'Triangular Well', "icon": TriangularIcon()},
            'para': { 'name':'Parabolic Potential',"icon": ParabolicIcon()},
            'cola': { 'name':'Coulombic Potential',"icon": CoulombIcon()},
            'expo': { 'name':'Exponential Potential',"icon": ExponentialIcon()},
            'gaus': { 'name':'Gaussian Potential',"icon": GaussianIcon()},
            'poly': { 'name':'Polynomial Potential',"icon": PolynomialIcon()}
        },
    },
    'degree': {
        'type': 'Number', 'default_value': '1', 'units': None, 'min': 1.0, 'max': 4.0, 'label': 'Enter degree of order:',
        'description': 'Degree of the polynomial function can only be 1,2,3,4'
    },
    'vmax': {
        'type': 'Number','default_value': 3,'units': 'eV','min': 0.1,'max': 40.0,'label': 'Maximum Barrier Height(Vmax)',
        'description': 'Maximum height a barrier can have in the well in eV.'
    },
    'vmin': {
        'type': 'Number','default_value': 0,'units': 'eV','min': -5.0,'max': 20.0,'label': 'Minimum Barrier Height(Vmin)',
         'description': 'Minimum height a barrier can have in the well in eV.'
    },
    'emax': {
        'type': 'Number','default_value': 5,'units': 'eV','min': 0.0,'max': 10.0,'label': 'Energy of particle',
        'description': 'Maximum energy carried by the particle over the barrier height.'
    },
    'vdepth': {
        'type': 'Number','default_value': 10,'units': 'eV','min': 0.0,'max': 40.0,'label': 'Coloumbic Well (-Vmax)',
        'description': 'Maximum depth of coulombic well'
    },
    'emin_c': {
        'type': 'Number','default_value': -10,'units': 'eV','min': -40.0,'max': 0.0,'label': 'Minimum Energy (Emin)',
        'description': 'Minimum energy of the particle in the well in eV.'
    },
    'emax_c': {
        'type': 'Number','default_value': 1,'units': 'eV','min': 0.0,'max': 20.0,'label': 'Particle energy',
        'description': 'Maximum energy carried by the particle Above the barrie'
    },
    'well_width': {
        'type': 'Number','default_value': 12,'units': 'A','min': 1.0,'max': 100.0,'label': 'Periodic cell Width(A)',
        'description': 'Periodicity of a single periodic cell in the well, distance from one atom to the next atom.'
    },
    'a': {
        'type': 'Number','default_value': 6,'units': 'A','min': 0.1,'max': 95.0,'label': 'Enter the a (A):',
        'description': 'Width in which the potential is maximum/minimum or changes from one extremal energy to another.'
    },
    'nodes': {
        'type': 'Number','default_value': '3','units': None,'min': 2.0,'max': 6.0,'label': 'Enter the number of nodes:',
        'description': 'Number of nodes in the sinosoidal well, i.e, number of times the wave intersects with x axis'
    },
    'mass': {
        'type': 'Number','default_value': '1.0','units': None,'min': 0.02,'max': 5.0,'label': 'Effective mass (m0)',
        'description': 'Effective mass describes the influence of the underlying crystal on the ability of an electron to propagate inside the parabolic conduction band. This is a relative number normalized against the free electron mass. Typical values are 0.067 for GaAs and 0.024 for InAs.  A completely free electron in space would take on the value of 1.0'
    },
    'b_expo': {
        'type': 'Number','default_value': '1','units': None,'min': -50.0,'max': 50.0,'label': 'b',
        'description': "Coefficient 'b' for exponential function.\n        Enter a number between -50 to 50"
    },
    'exp_shift': {
        'type': 'Number','default_value': '0','units': None,'min': 0.0,'max': 100.0,'label': 'shift(%)',
        'description': 'Percentage you want to shift the function within the well'
    },
    'c_gaus': {
        'type': 'Number','default_value': '1','units': None,'min': -50.0,'max': 50.0,'label': 'c',
        'description': "Coefficient 'c' for Gaussian function. Enter a number between -50 to 50"
    },
    'gaus_shift': {
        'type': 'Number','default_value': '0','units': None,'min': 0.0,'max': 100.0,'label': 'shift(%)',
        'description': 'Percentage you want to shift the function within the well'
    },
    'a_poly2': {
        'type': 'Number','default_value': '1','units': None,'min': -10000.0,'max': 10000.0,'label': 'a',
        'description': "Coefficient 'a' for second order polynomial function. Enter a number between -10000 to 10000"
    },
    'b_poly2': {
        'type': 'Number','default_value': '1','units': None,'min': None,'max': None,'label': 'b',
        'description': "Coefficient 'b' for second order polynomial function"
    },
    'poly2_shift': {
        'type': 'Number','default_value': '0','units': None,'min': 0.0,'max': 100.0,'label': 'shift(%)',
        'description': 'enter the percentage you want to shift the function within the well'
    },
    'a_poly3': {
        'type': 'Number','default_value': '1','units': None,'min': None,'max': None,'label': 'a',
        'description': "Coefficient 'a' for third order polynomial function"
    },
    'b_poly3': {
        'type': 'Number','default_value': '1','units': None,'min': None,'max': None,'label': 'b',
        'description': "Coefficient 'b' for third order polynomial function"
    },
    'c_poly3': {
        'type': 'Number','default_value': '1','units': None,'min': None,'max': None,'label': 'c',
        'description': "Coefficient 'c' for third order polynomial function"
    },
    'poly3_shift': {
        'type': 'Number','default_value': '0','units': None,'min': 0.0,'max': 100.0,'label': 'shift(%)',
        'description': 'Percentage you want to shift the function within the well'
    },
    'a_poly4': {
        'type': 'Number','default_value': '1','units': None,'min': None,'max': None,'label': 'a',
        'description': "Coefficient 'a' for fourth order polynomial function"
    },
    'b_poly4': {
        'type': 'Number','default_value': '1','units': None,'min': None,'max': None,'label': 'b',
        'description': "Coefficient 'b' for fourth order polynomial function"
    },
    'c_poly4': {
        'type': 'Number','default_value': '1','units': None,'min': None,'max': None,'label': 'c',
        'description': "Coefficient 'c' for fourth order polynomial function"
    },
    'd_poly4': {
        'type': 'Number','default_value': '1','units': None,'min': None,'max': None,'label': 'd',
        'description': "Coefficient 'd' for fourth order polynomial function"
    },
    'poly4_shift': {
        'type': 'Number','default_value': '0','units': None,'min': 0.0,'max': 100.0,'label': 'shift(%)',
        'description': 'Percentage you want to shift the function within the well'
    }
} 

## Defining Layout

In [11]:
LAYOUT = { 'input': 
  { 'type': 'group','label': 'Periodic Potential Details','enable': None, 'layout': 'horizontal','children': [
    { 'type': 'choice', 'id': 'pot_type', 'label': 'Select the type of periodic potential', 'enable': None},
    { 'type': 'number', 'id': 'degree', 'label': 'Enter degree of order:', 'enable': [
        {'operand': 'pot_type','operator': '==','value': 'poly','condition': ''}
    ]},
    { 'type': 'tab', 'label': 'Well Details', 'enable': None, 'layout': 'horizontal', 'children': [
        { 'type': 'group', 'label': 'Energy Details', 'enable': None, 'layout': 'horizontal', 'children': [
          {'type': 'number','id': 'vmax','label': 'Maximum Barrier Height(Vmax)','enable': [
            {'operand': 'pot_type','operator': 'in','value': 'KP,Tri,para,Sine,expo,gaus,poly','condition': ''}
          ]}, 
          {'type': 'number', 'id': 'vmin', 'label': 'Minimum Barrier Height(Vmin)', 'enable': [
            {'operand': 'pot_type','operator': 'in','value': 'KP,Tri,para,Sine,expo,gaus,poly','condition': ''}

          ]}, 
          { 'type': 'number', 'id': 'emax', 'label': 'Energy of  particle over the barrier:', 'enable': [
            {'operand': 'pot_type','operator': 'in','value': 'KP,Tri,para,Sine,expo,gaus,poly','condition': ''}
          ]}, 
          { 'type': 'number', 'id': 'vdepth', 'label': 'Depth of Coloumbic Well (-Vmax)', 'enable': [
            { 'operand': 'pot_type', 'operator': '==', 'value': 'cola', 'condition': '' }
          ]}, 
          { 'type': 'number', 'id': 'emin_c', 'label': 'Minimum Carrier Energy (Emin)', 'enable': [
            { 'operand': 'pot_type', 'operator': '==', 'value': 'cola', 'condition': '' }
          ]}, 
          { 'type': 'number', 'id': 'emax_c', 'label': 'Particle energy above the barrier', 'enable': [
            { 'operand': 'pot_type', 'operator': '==', 'value': 'cola', 'condition': '' }
          ]}
        ]}, 
        { 'type': 'group', 'label': 'Well Geometry', 'enable': None, 'layout': 'horizontal', 'children': [
          { 'type': 'number', 'id': 'well_width', 'label': 'Width of single periodic cell (A)', 'enable': [
            {'operand': 'pot_type','operator': 'in','value': 'KP,Tri,para,Sine,expo,gaus,poly,cola','condition': ''}
          ]}, 
          { 'type': 'number', 'id': 'a', 'label': 'Enter the a (A):', 'enable': [
            { 'operand': 'pot_type', 'operator': 'in', 'value': 'KP,Tri,cola', 'condition': '||' }, 
          ]}, 
          { 'type': 'number', 'id': 'nodes', 'label': 'Enter the number of nodes:', 'enable': [
            { 'operand': 'pot_type', 'operator': '==', 'value': 'Sine', 'condition': '' }
          ]}, 
          { 'type': 'number', 'id': 'mass', 'label': 'Effective mass of the particle (m0)', 'layout': 'horizontal', 'enable': None }, 
          { 'type': 'group', 'label': 'exponential coefficients', 'enable': 
            [
              { 'operand': 'pot_type', 'operator': '==', 'value': 'expo'}
            ], 'children':         
            [
              { 'type': 'number', 'id': 'b_expo', 'label': 'b', 'enable': None },
              { 'type': 'number', 'id': 'exp_shift', 'label': 'shift(%)', 'enable': None }
            ]
          }, 
          { 'type': 'group', 'label': 'Gaussian coefficients', 'enable': 
            [
              { 'operand': 'pot_type', 'operator': '==', 'value': 'gaus'}
            ], 'layout': 'horizontal', 'children': 
            [
              { 'type': 'number', 'id': 'c_gaus', 'label': 'c', 'enable': None }, 
              { 'type': 'number', 'id': 'gaus_shift', 'label': 'shift(%)', 'enable': None } 
            ]
          }, 
          { 'type': 'group', 'label': 'second order polynomial coefficients', 'enable': 
            [
              { 'operand': 'pot_type', 'operator': '==', 'value': 'poly'},
              { 'operand': 'degree', 'operator': '==', 'value': 2}
            ], 'layout': 'horizontal', 'children': [
              { 'type': 'number', 'id': 'a_poly2', 'label': 'a', 'enable': None }, 
              { 'type': 'number', 'id': 'b_poly2', 'label': 'b', 'enable': None }, 
              { 'type': 'number', 'id': 'poly2_shift', 'label': 'shift(%)', 'enable': None } 
            ]
          }, 
          { 'type': 'group', 'label': 'third order polynomial coefficients', 'enable': 
            [
              { 'operand': 'pot_type', 'operator': '==', 'value': 'poly'},
              { 'operand': 'degree', 'operator': '==', 'value': 3}
            ], 'layout': 'horizontal', 'children': 
            [
              { 'type': 'number', 'id': 'a_poly3', 'label': 'a', 'enable': None }, 
              { 'type': 'number', 'id': 'b_poly3', 'label': 'b', 'enable': None }, 
              { 'type': 'number', 'id': 'c_poly3', 'label': 'c', 'enable': None }, 
              { 'type': 'number', 'id': 'poly3_shift', 'label': 'shift(%)', 'enable': None } 
            ]
          }, 
          { 'type': 'group', 'label': 'fourth order polynomial coefficients', 'enable': 
            [
              { 'operand': 'pot_type', 'operator': '==', 'value': 'poly'},
              { 'operand': 'degree', 'operator': '==', 'value': 4}
            ], 'layout': 'horizontal', 'children': [
              { 'type': 'number', 'id': 'a_poly4', 'label': 'a', 'enable': None }, 
              { 'type': 'number', 'id': 'b_poly4', 'label': 'b', 'enable': None }, 
              { 'type': 'number', 'id': 'c_poly4', 'label': 'c', 'enable': None }, 
              { 'type': 'number', 'id': 'd_poly4', 'label': 'd', 'enable': None }, 
              { 'type': 'number', 'id': 'poly4_shift', 'label': 'shift(%)', 'enable': None } 
            ]
          } 
        ]}
    ]}
  ]}
}
#LAYOUT

## Creating the settings panel

In [12]:
url_sim = "https://nanohub.org/api/tools"
Settings = AppBuilder.Settings(
    Project,
    Component, 
    SETTINGS,
    url=url_sim,
    toolname = TOOLNAME,
    layout = LAYOUT['input']
)
Settings.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False},
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN,"newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS,"newState": '$e'},
]
Settings.content.events["click"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True}
]

Settings.content.events["onStatusChange"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_STATUS,"newState": "$e.target.value"}
]
Settings.content.events["onSuccess"]=[
  { "type": "stateChange", "modifies": "open_plot","newState":{'option8a':'primary'} },
  { "type": "propCall2", "calls": "loadXYDual","args": ["self", "['^EK([1-9]|10)$']", "['^eig_energy_min','^potential','norm_wave_fcn_min']"] },
  { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False },  
  { "type": "stateChange", "modifies": "open_results","newState": True},
  { "type": "stateChange", "modifies": "open_params","newState": False},
]

## Creating the output options

In [13]:
RapptureBuilder.loadXY(Project, Component)
RapptureBuilder.loadXYDual(Project, Component)
RapptureBuilder.loadSequence(Project, Component)
RESULTS = {
    "option8a" : {
        'title' : 'Dispersion & Wavefunction (Min)',
        'action' : { "type": "propCall2", "calls": "loadXYDual","args": ["self", "['^EK([1-9]|10)$']", "['^eig_energy_min','^potential','norm_wave_fcn_min']", "{'showlegend':false}"] },
    },    
    "option8b" : {
        'title' : 'Dispersion & Wavefunction (Max)',
        'action' : { "type": "propCall2", "calls": "loadXYDual","args": ["self", "['^EK([1-9]|10)$']", "['^eig_energy_max','^potential','norm_wave_fcn_max']", "{'showlegend':false}"] },
    },      
    #"option7a" : {
    #    'title' : 'Eigen Energy and Wave function (Min)',
    #    'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['^eig_energy_min','wave_fcn_min','pot']", "{'showlegend':false}"] },
    #},
    #"option7b" : {
    #    'title' : 'Eigen Energy and Wave function (Max)',
    #    'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['^eig_energy_max','wave_fcn_max','pot']", "{'showlegend':false}"] },
    #},
    "option3" : {
        'title' : 'Reduced Dispersion Relation',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['^EK([1-9]|10)$']"] },
    },
    "option4" : {
        'title' : 'Expanded Dispertion Relation',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['^EK1([1-9]|10)$']"] },
    },
    "option6" : {
        'title' : 'Reduced Dispersion with Eff. Masses',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['^EKeff','^EK_pp']", "{'showlegend':false}"] },
    },
    "option9" : {
        'title' : 'Density of States',
        'action' : { 
            "type": "propCall2", 
            "calls": "loadXYDual",
            "args": ["self", "['^EK([1-9]|10)$']", "['^DOS_1D$', '^EK_dos']", "{'xaxis':{'type':'log'},'showlegend':false}"]
        },
    },    
    "option5" : {
        'title' : 'Expanded Dispersion vs Free Dispersion',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['^EKcomp','^EKfree']", "{'showlegend':false}"] },
    },
    "option2" : {
        'title' : 'Allowed Bands',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['^Band limit', '^FE limit','^Ebands']", "{'showlegend':false}"] },
    },

    "option1" : {
        'title' : 'Energy functional vs Energy',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['^TvsE']"] },
    },
}
    
'''
    "option4":{
        "alt" : "Expanded Dispertion Relation",
        "label" : "Expanded Dispertion",
        "action" : function(){ self.displayParameter('option4') },
    },

'''    

Results = AppBuilder.Results( 
    Component,
    results = RESULTS,
    onClick = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True }],
    onLoad = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False }],
)


## Creating Colorschema and adding nanoHUB logo

In [14]:
Theme = MaterialBuilder.DefaultTheme(
    primary_color = '#699FBB',
    secondary_color = '#f1f1f1',
    primary_bg = '#FFFFFF',
    secondary_bg = '#dbeaf0',
    default_button = 'rgba(255, 255, 255, 0.87)',
    primary_button = 'rgba(255, 255, 255, 0.87)',
    secondary_button = 'rgba(0, 0, 0, 0.87)',
    default_button_bg = 'rgb(63, 162, 192)',
    primary_button_bg = 'rgba(0, 0, 0, 0.65)',
    secondary_button_bg = 'rgba(0, 0, 0, 0.12)',    
    drawer_position = "relative"
)
ThemeProvider = MaterialBuilder.ThemeProvider( Component, Theme )
Theme['components']['MuiDrawer']['styleOverrides']['paper']['margin-top'] = '0px'
Theme['components']['MuiDrawer']['styleOverrides']['paper']['height'] = '100%'
                                              
AppBar = MaterialBuilder.AppBar(
    state="DrawerIsVisible", 
    title="Kronig Penney Lab"
)

logo = t.TeleportElement(t.TeleportContent(elementType="img"))
logo.content.attrs["width"] = "120"
logo.content.attrs["src"] = "https://nanohub.org/app/site/media/images/PressKit/nanoHUB_logo_color.jpg"
AppBar.content.children[0].addContent(logo)

## Creating Expansion panels

In [15]:

ExpansionPanel = MaterialBuilder.ExpansionPanel(
    title="Results", 
    content=[Results],
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_results"
      }    
    },     
)
ExpansionPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_results","newState": "$toggle"}]
Drawer.addContent(ExpansionPanel)

ParametersPanel = MaterialBuilder.ExpansionPanel(
    title="Parameters", 
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_params"
      }    
    }, 
    content=[Settings]
)
ParametersPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_params","newState": "$toggle"}]
Drawer.addContent(ParametersPanel)


## Creating reset button

In [16]:
Gridt = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridt.content.attrs["color"] = "secondary"
Gridt.content.attrs["container"] = True
Gridt.content.attrs["direction"] = "column"
resetSettings = {}
for k,v in SETTINGS.items():
    resetSettings[k] = v["default_value"]

Buttontt= MaterialBuilder.Button(
      title = "Reset Setting", 
      variant = "text", 
      style = {'backgroundColor':'#999999', 'borderRadius' : '0px', 'minHeight':'40px', 'color':'rgba(255, 255, 255, 0.87)'}, 
      onClickButton=[{ "type": "stateChange", "modifies": "parameters","newState": resetSettings}]
)
Gridt.addContent(Buttontt)
Drawer.addContent(Gridt)

## Creating ploting component

In [17]:
BasePlot = PlotlyBuilder.BasePlot( Project, Component, style_state="AppBarStyle")


In [18]:
ThemeProvider.addContent(Login)


## Assembling the app

In [19]:
Gridh = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridh.content.attrs["container"] = True
Gridh.content.attrs["direction"] = "row" 
Gridh.addContent(Drawer)
Gridh.addContent(BasePlot)  

Gridv = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridv.content.attrs["container"] = True
Gridv.content.attrs["direction"] = "column"
Gridv.addContent(AppBar)
Gridv.addContent(Gridh)


ThemeProvider.addContent(Gridv)
ThemeProvider.addContent(Loader)
ThemeProvider.addContent(ErrorMessage)

Component.addNode(ThemeProvider)
Project.buildReact(
    TOOLNAME.upper().replace("_","")+".HTML", 
    copy_libraries=False, 
    run_uidl="redirect", 
    jupyter_notebook_url=jupyter_notebook_url 
);

building HTML 
done!
https://proxy.nanohub.org/weber/2163531/4zR6ixKMr1itlvZR/23/uidl/KRONIGPENNEY.HTML/redirect/nanohub-uidl/examples/tools/KRONIGPENNEY.HTML


## Displaying the app in jupyter

In [20]:
from nanohubuidl.ipywidgets import buildWidget
BasePlot.content.style = {
  "position": "absolute",
  "height": "700px",
  "width": "100%",
}
Drawer.content.style = {
  "height": "700px",
}
MyComponentWidget = buildWidget(Project, jupyter_axios=True, debugger=True, verbose=False);

In [21]:
a = MyComponentWidget()
a

HTML(value="<link rel='stylesheet' href='https://fonts.googleapis.com/icon?family=Material+Icons'/><style>.p-W…

<IPython.core.display.Javascript object>

PeriodicPotentialLabWidget(config={'displayModeBar': True, 'responsive': True}, data=[{'x': [], 'y': []}], par…